# Dependenicas e Variaveis Nessesarios

In [2]:
from langchain_ollama import ChatOllama

from langchain_core.messages import SystemMessage, HumanMessage 

from langchain_core.prompts import (SystemMessagePromptTemplate,
                                    HumanMessagePromptTemplate,
                                    MessagesPlaceholder,
                                    PromptTemplate,
                                    ChatPromptTemplate)


from langchain_core.output_parsers import StrOutputParser

from langchain_core.runnables.history import RunnableWithMessageHistory

from langchain_community.chat_message_histories import SQLChatMessageHistory as hist

from langchain_ollama import OllamaEmbeddings

import faiss

from langchain_community.docstore.in_memory import InMemoryDocstore

from langchain_community.vectorstores import FAISS

from dotenv import load_dotenv
load_dotenv("./../.env")

parse = StrOutputParser()
base_url = "127.0.0.1:11434"
# chatbot = "deepseek-r1:1.5b"
chatbot = "llama3.2:latest"

# Embedding

In [3]:
embed =  OllamaEmbeddings(
    model="nomic-embed-text:latest",
)

In [6]:
text = "LangChain is the framework for building context-aware reasoning applications"

index = faiss.IndexFlatL2(len(embed.embed_query("hello world")))

vector_store = FAISS(
    embedding_function=embed,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)


In [ ]:
from uuid import uuid4

from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocalate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]
uuids = [str(uuid4()) for _ in range(len(documents))]

vector_store.add_documents(documents=documents, ids=uuids)

In [ ]:
retriever = vector_store.as_retriever(search_type = "mmr" , search_kwargs= {'k': 2})

retriever.invoke("Stealing from the bank is a crime")

# todo :

    # Funcao que pega chatprompt que vai pra llm e mandar uma copia para embed

    # Funcao que embeddar a resposta da llm

    # funcao que grave para vector store (faiss)

In [6]:
vector_store.save_local("faiss_index")

In [7]:
chatter = ChatOllama(
    base_url=base_url,
    model=chatbot
)
# chatter.invoke("faz print de hello world em php").pretty_print()

chatter.invoke("explain latent spaces using simple terminology.").pretty_print()

================================== Ai Message ==================================

Imagine you're trying to describe a picture of a cat. You could say things like "the cat has whiskers" or "the cat is black and white." But, if someone asks you what the whiskers are made of, or how the colors got there, you might not be able to explain it in simple terms.

That's kind of like what happens when we try to understand images or videos. Our brains can pick up on patterns and features, but sometimes those patterns are hard to put into words.

Latent spaces are a way to represent those hidden patterns and features as numbers. Think of them like a secret code that lets computers understand the underlying structure of an image or video.

Imagine you have a bunch of different pictures, each with its own unique features (like whiskers, eyes, nose, etc.). When we put all these pictures together into one big database, it's hard to see what makes each picture special. But if we use latent spaces, we c

In [7]:
# prompt template
# human named templates are used to define messages sent from the user
_human =  HumanMessagePromptTemplate.from_template("{input}")
#system named templates are used to define  messages that define the ai role
_system = SystemMessagePromptTemplate.from_template("Your name is Angel")
#
_messages = [_human, MessagesPlaceholder(variable_name='history'), _system]

_prompt = ChatPromptTemplate(messages=_messages)

chain =  _prompt | chatter | parse 

In [ ]:
# persistance is maintainded by langchain through session ids this can be called back through sql queries
connection_string = 'sqlite:///vector_store/chat_history.db'
def get_session_history(session_id):
    return  hist(session_id, connection_string)

runnable_history = RunnableWithMessageHistory(chain, get_session_history, input_messages_key='input', history_messages_key='history')

In [ ]:
def chat_with_llm(session_id, input):
    output = runnable_history.invoke(
        {'input' : input},
        config={'configurable' : {'session_id' : session_id}}
    )
    
    return output

In [ ]:
user_id = "celo"
chat_with_llm(user_id, "explica porque o brasil e tao legal")

In [8]:
# getting history
history = get_session_history(user_id)

NameError: name 'get_session_history' is not defined

In [ ]:
history.get_messages()

In [ ]:
history.clear()

In [ ]:
#prompt templates
#templates that can help generate prompts dynamically

question_template = HumanMessagePromptTemplate.from_template("explain {topic}, using {level} level language")
system_template = SystemMessagePromptTemplate.from_template("you are a {prof}. describe things using ideas related to your profession.")
analysis_prompt = ChatPromptTemplate.from_template('''Answer as though you were a small child.
                                                   You need to tell me how difficult the text you recieve is to understand.
                                                   Answer in one sentence only.
                                                   Analyze the following text: {response}
                                                   ''')

#chat template
messages_template = [question_template, system_template]
chat_template = ChatPromptTemplate(messages_template)







In [ ]:
#using chains
# chains change how things get invoked. 
#chain uses | to bind a chat template to a model
chain = chat_template | chatter | StrOutputParser()


In [ ]:
#chaining Runnables.
# chain -> new chain

In [ ]:
#composed chain
# {"variable" : runnable} | next_runnable

# check_bot = {"response" : chain} | analysis_prompt | checker | parse

# check_chain_response = check_bot.invoke({  "level" : "pdh",  "prof" : "biologist", "topic":"soil"})
# print(check_chain_response)

In [ ]:
from langchain_core.runnables import RunnableLambda

output parsers are classes that structure the output of information from an llm. this is accomplished through the use of pydantic -base model, field- and PydanticOutputParser. build a class that abstracts the task. you can then use typed variables with fields whose descriptions are natural language tasks. these class properties are passed into the system prompt as formating instructions along with the query. 

In [8]:
from langchain_community.document_loaders import PyMuPDFLoader

file_path = r"D:\Dev\langchain\lang\steamlit\splitter\documents\artigo.pdf"
loader = PyMuPDFLoader(file_path)

In [9]:
docs = loader.load()
print(len(docs))

4


In [10]:
import pprint

pprint.pp(docs[0].metadata)

{'producer': 'LibreOffice 24.2',
 'creator': 'Writer',
 'creationdate': '2025-03-04T19:50:50-03:00',
 'source': 'D:\\Dev\\langchain\\lang\\steamlit\\splitter\\documents\\artigo.pdf',
 'file_path': 'D:\\Dev\\langchain\\lang\\steamlit\\splitter\\documents\\artigo.pdf',
 'total_pages': 4,
 'format': 'PDF 1.7',
 'title': '',
 'author': '',
 'subject': '',
 'keywords': '',
 'moddate': '',
 'trapped': '',
 'modDate': '',
 'creationDate': "D:20250304195050-03'00'",
 'page': 0}


In [11]:
import os

pdfs = []
for root, dirs, files in os.walk("./steamlit/splitter/documents"):
    print(root, dirs, files)
    for file in files:
        if file.endswith(".pdf"):
            pdfs.append(os.path.join(root,file))

./steamlit/splitter/documents [] ['artigo.pdf', 'test-IA.pdf']


In [12]:
docs = []
for pdf in pdfs:
    loader = PyMuPDFLoader(pdf)
    temp = loader.load()
    docs.extend(temp)
    
    print(temp)
    
    
    

[Document(metadata={'producer': 'LibreOffice 24.2', 'creator': 'Writer', 'creationdate': '2025-03-04T19:50:50-03:00', 'source': './steamlit/splitter/documents\\artigo.pdf', 'file_path': './steamlit/splitter/documents\\artigo.pdf', 'total_pages': 4, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': "D:20250304195050-03'00'", 'page': 0}, page_content='Sistema de LLM RAG com Dados \nEmpresariais: Integração de Langchain, React \ne Django para Comunicação de API\nResumo\nEste artigo descreve o desenvolvimento de um sistema de Resposta Através de Geração (RAG) \nutilizando modelos de linguagem (LLMs) com dados específicos de uma empresa. A solução \nemprega Langchain para integração de modelos de linguagem com fontes de dados, React para a \ninterface de usuário e Django para criar a API de comunicação entre o front-end e o back-end. O \nobjetivo deste sistema é permitir que os usuários possam consulta

In [13]:
def format_text(docs):
    return '\n\n'.join([doc.page_content for doc in docs])


context = format_text(docs)

In [14]:
# check of token size of docs
print(context)

Sistema de LLM RAG com Dados 
Empresariais: Integração de Langchain, React 
e Django para Comunicação de API
Resumo
Este artigo descreve o desenvolvimento de um sistema de Resposta Através de Geração (RAG) 
utilizando modelos de linguagem (LLMs) com dados específicos de uma empresa. A solução 
emprega Langchain para integração de modelos de linguagem com fontes de dados, React para a 
interface de usuário e Django para criar a API de comunicação entre o front-end e o back-end. O 
objetivo deste sistema é permitir que os usuários possam consultar dados empresariais específicos 
de maneira eficiente e inteligente, utilizando um modelo de linguagem capaz de acessar, interpretar 
e gerar respostas personalizadas baseadas em informações empresariais. A arquitetura proposta é 
escalável, modular e integrada, permitindo a fácil expansão e adaptação conforme as necessidades 
do negócio.
1. Introdução
Nos últimos anos, os Modelos de Linguagem de Grande Escala (LLMs) têm demonstrado grande 
pote

In [18]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o-mini")

In [19]:
len(encoding.encode(context))

1729

In [15]:
from langchain_text_splitters import RecursiveCharacterTextSplitter


text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 100)

chunks = text_splitter.split_documents(docs)

In [16]:
pprint.pp(chunks)

[Document(metadata={'producer': 'LibreOffice 24.2', 'creator': 'Writer', 'creationdate': '2025-03-04T19:50:50-03:00', 'source': './steamlit/splitter/documents\\artigo.pdf', 'file_path': './steamlit/splitter/documents\\artigo.pdf', 'total_pages': 4, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': "D:20250304195050-03'00'", 'page': 0}, page_content='Sistema de LLM RAG com Dados \nEmpresariais: Integração de Langchain, React \ne Django para Comunicação de API\nResumo\nEste artigo descreve o desenvolvimento de um sistema de Resposta Através de Geração (RAG) \nutilizando modelos de linguagem (LLMs) com dados específicos de uma empresa. A solução \nemprega Langchain para integração de modelos de linguagem com fontes de dados, React para a \ninterface de usuário e Django para criar a API de comunicação entre o front-end e o back-end. O \nobjetivo deste sistema é permitir que os usuários possam consulta

In [ ]:
len(encoding.encode(chunks[0].page_content))

[104264,
 334,
 451,
 19641,
 460,
 2971,
 452,
 141520,
 793,
 24156,
 430,
 6477,
 276,
 25,
 89184,
 23267,
 1747,
 334,
 27830,
 13289,
 11,
 6184,
 793,
 68,
 121538,
 1209,
 146498,
 334,
 10328,
 198,
 179818,
 198,
 26237,
 45084,
 161800,
 737,
 293,
 37784,
 334,
 1713,
 12767,
 334,
 2317,
 22686,
 355,
 180246,
 334,
 499,
 31798,
 350,
 49,
 2971,
 8,
 793,
 2056,
 52248,
 39564,
 334,
 108504,
 350,
 7454,
 25153,
 8,
 452,
 24315,
 97292,
 334,
 3030,
 13596,
 13,
 355,
 71507,
 793,
 7175,
 42989,
 27830,
 13289,
 1209,
 144517,
 334,
 39564,
 334,
 108504,
 452,
 115142,
 334,
 24315,
 11,
 6184,
 1209,
 261,
 793,
 11132,
 334,
 67663,
 319,
 121538,
 1209,
 44999,
 261,
 10328,
 334,
 63586,
 3853,
 293,
 5567,
 18507,
 319,
 293,
 1602,
 18507,
 13,
 532,
 793,
 61436,
 4330,
 28924,
 12767,
 1212,
 59290,
 661,
 1994,
 73354,
 90153,
 73071,
 24315,
 8430,
 6477,
 276,
 97292,
 793,
 613,
 39477,
 75394,
 319,
 80759,
 11,
 54002,
 1713,
 23255,
 334,
 108504,
 389

In [17]:
from langchain_ollama import OllamaEmbeddings

import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
base_url = "127.0.0.1:11434"

In [18]:
embeddings = OllamaEmbeddings(model="nomic-embed-text:latest", base_url=base_url)

In [20]:
vector = embeddings.embed_query("hi")

In [33]:
len(vector)

768

In [21]:
index = faiss.IndexFlatL2(len(vector))
index.ntotal, index.d

(0, 768)

In [22]:
vector_store= FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

In [23]:
ids = vector_store.add_documents(documents=chunks)

In [24]:
q = "what is RAG?"
docs = vector_store.search(query=q,k=5,search_type="similarity")

In [25]:
docs

[Document(id='f6707d51-3314-4440-a623-a87353670590', metadata={'producer': 'LibreOffice 24.2', 'creator': 'Writer', 'creationdate': '2025-03-04T19:50:50-03:00', 'source': './steamlit/splitter/documents\\artigo.pdf', 'file_path': './steamlit/splitter/documents\\artigo.pdf', 'total_pages': 4, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': "D:20250304195050-03'00'", 'page': 0}, page_content='Sistema de LLM RAG com Dados \nEmpresariais: Integração de Langchain, React \ne Django para Comunicação de API\nResumo\nEste artigo descreve o desenvolvimento de um sistema de Resposta Através de Geração (RAG) \nutilizando modelos de linguagem (LLMs) com dados específicos de uma empresa. A solução \nemprega Langchain para integração de modelos de linguagem com fontes de dados, React para a \ninterface de usuário e Django para criar a API de comunicação entre o front-end e o back-end. O \nobjetivo deste sistema

In [26]:
from langchain_ollama import OllamaEmbeddings

import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore
base_url = "127.0.0.1:11434"

In [34]:
embeddings = OllamaEmbeddings(model="nomic-embed-text:latest", base_url=base_url)
db_path = r"D:\Dev\langchain\lang\steamlit\vector_db"
db_name = "cerebro"
vector_store.save_local(folder_path=db_path,index_name=db_name)

In [38]:
_vectore_store = FAISS.load_local(db_path, embeddings, db_name,allow_dangerous_deserialization=True)

In [40]:
q = "what is RAG?"
docs = _vectore_store.search(query=q,k=5,search_type="similarity")
docs

[Document(id='dce52bea-0703-4191-80e1-29836c676d6d', metadata={'producer': 'LibreOffice 24.2', 'creator': 'Writer', 'creationdate': '2025-03-04T19:50:50-03:00', 'source': 'D:\\Dev\\langchain\\lang\\steamlit\\splitter\\documents\\artigo.pdf', 'file_path': 'D:\\Dev\\langchain\\lang\\steamlit\\splitter\\documents\\artigo.pdf', 'total_pages': 4, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': "D:20250304195050-03'00'", 'page': 0}, page_content='Sistema de LLM RAG com Dados \nEmpresariais: Integração de Langchain, React \ne Django para Comunicação de API\nResumo\nEste artigo descreve o desenvolvimento de um sistema de Resposta Através de Geração (RAG) \nutilizando modelos de linguagem (LLMs) com dados específicos de uma empresa. A solução \nemprega Langchain para integração de modelos de linguagem com fontes de dados, React para a \ninterface de usuário e Django para criar a API de comunicação entre 

In [41]:
retriever = _vectore_store.as_retriever(search_type = "mmr", search_kwargs= {'k':3, 'fetch_k': 20, 'lambda_mult': 1})
retriever.invoke(q)

[Document(id='dce52bea-0703-4191-80e1-29836c676d6d', metadata={'producer': 'LibreOffice 24.2', 'creator': 'Writer', 'creationdate': '2025-03-04T19:50:50-03:00', 'source': 'D:\\Dev\\langchain\\lang\\steamlit\\splitter\\documents\\artigo.pdf', 'file_path': 'D:\\Dev\\langchain\\lang\\steamlit\\splitter\\documents\\artigo.pdf', 'total_pages': 4, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '', 'trapped': '', 'modDate': '', 'creationDate': "D:20250304195050-03'00'", 'page': 0}, page_content='Sistema de LLM RAG com Dados \nEmpresariais: Integração de Langchain, React \ne Django para Comunicação de API\nResumo\nEste artigo descreve o desenvolvimento de um sistema de Resposta Através de Geração (RAG) \nutilizando modelos de linguagem (LLMs) com dados específicos de uma empresa. A solução \nemprega Langchain para integração de modelos de linguagem com fontes de dados, React para a \ninterface de usuário e Django para criar a API de comunicação entre 

In [42]:
llm = chatter

In [44]:

_prompt = """Anwer from context provided only. If you don't have the information to answer a question, say 'I don't know'. 
### Context:
{context}
### Question:
{question}
### Answer"""

def d(docs):
   return '\n\n'.join([doc.page_content for doc in docs])  

print(d(docs))    


Sistema de LLM RAG com Dados 
Empresariais: Integração de Langchain, React 
e Django para Comunicação de API
Resumo
Este artigo descreve o desenvolvimento de um sistema de Resposta Através de Geração (RAG) 
utilizando modelos de linguagem (LLMs) com dados específicos de uma empresa. A solução 
emprega Langchain para integração de modelos de linguagem com fontes de dados, React para a 
interface de usuário e Django para criar a API de comunicação entre o front-end e o back-end. O 
objetivo deste sistema é permitir que os usuários possam consultar dados empresariais específicos 
de maneira eficiente e inteligente, utilizando um modelo de linguagem capaz de acessar, interpretar 
e gerar respostas personalizadas baseadas em informações empresariais. A arquitetura proposta é 
escalável, modular e integrada, permitindo a fácil expansão e adaptação conforme as necessidades 
do negócio.
1. Introdução
Nos últimos anos, os Modelos de Linguagem de Grande Escala (LLMs) têm demonstrado grande

Nos 

In [28]:
context = d(docs)
context

NameError: name 'd' is not defined

In [45]:
from langchain_core.runnables import RunnablePassthrough

_prompt = """Anwer from context provided only. If you don't have the information to answer a question, say 'I don't know'. 
### Context:
{context}
### Question:
{question}
### Answer"""

system = SystemMessagePromptTemplate.from_template("You're a helpful assistance ready to answer in a calm and hospitible tone.")
human = HumanMessagePromptTemplate.from_template(_prompt)

messages = [system, human]

prompt = ChatPromptTemplate(messages)

rag_chain = (
    {"context" : retriever|d, "question": RunnablePassthrough()}
    | prompt 
    | llm 
    | StrOutputParser()
)

In [46]:
q

'what is RAG?'

In [47]:
response = rag_chain.invoke(q)

In [88]:
pprint.pp(response)

('RAG significa Retrieval-Augmented Generation, um sistema que combina a '
 'recuperação de informações de bases de dados específicas com a capacidade de '
 'geração de conteúdo de Modelos de Linguagem (LLMs).')
